In [35]:
import sys
 
sys.path.append('../')

In [37]:
import openai
import json

from src.constants import API_KEYS
from src.utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
openai.api_key = API_KEYS["cybor"][0]

In [12]:

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation(messages, functions):
    # Step 1: send the conversation and available functions to GPT
    messages = messages
    functions = functions
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return response, second_response

TypeError: run_conversation() missing 2 required positional arguments: 'messages' and 'functions'

In [19]:
content = """Assume you are Joshua, an experienced data analyst. I will give you a detailed description of objects in an image and will ask you to complete a few tasks:

Description: "There is a 256 by 256 aerial image. One building is located at x = 67 and y = 8, in the top left of the image, with a size = 5775 pixels. One building is located at x = 114 and y = 59, in the center towards the top of the image, with a size = 3185 pixels. One building is located at x = 188 and y = 83, in the top right of the image, with a size = 2565 pixels. One building is located at x = 196 and y = 239, in the bottom right of the image, with a size = 2337 pixels. One building is located at x = 188 and y = 162, in the bottom right of the image, with a size = 6160 pixels. One building is located at x = 145 and y = 207, in the center towards the bottom of the image, with a size = 8374 pixels. One building is located at x = 153 and y = 41, in the center towards the top of the image, with a size = 3120 pixels. One building is located at x = 199 and y = 2, in the top right of the image, with a size = 7884 pixels. One building is located at x = 222 and y = 60, in the top right of the image, with a size = 2300 pixels. One small car is located at x = 30 and y = 34, in the top left of the image, with a size = 182 pixels. One small car is located at x = 38 and y = 50, in the top left of the image, with a size = 168 pixels. One small car is located at x = 56 and y = 70, in the top left of the image, with a size = 240 pixels. One small car is located at x = 86 and y = 83, in the top left of the image, with a size = 288 pixels."

Task 1: Determine spatial relations between objects. Task 2: Extract high-level spatial patterns between the objects, such as direction, clustering, dispersion, encirclement, interposition, etc. Task 3: Generate one or more independent image captions with high-level significant spatial patterns. Each starts with "CAP", with no longer than 32 tokens.
"""

In [29]:
content.replace("\n", " ")

'Assume you are Joshua, an experienced data analyst. I will give you a detailed description of objects in an image and will ask you to complete a few tasks:  Description: "There is a 256 by 256 aerial image. One building is located at x = 67 and y = 8, in the top left of the image, with a size = 5775 pixels. One building is located at x = 114 and y = 59, in the center towards the top of the image, with a size = 3185 pixels. One building is located at x = 188 and y = 83, in the top right of the image, with a size = 2565 pixels. One building is located at x = 196 and y = 239, in the bottom right of the image, with a size = 2337 pixels. One building is located at x = 188 and y = 162, in the bottom right of the image, with a size = 6160 pixels. One building is located at x = 145 and y = 207, in the center towards the bottom of the image, with a size = 8374 pixels. One building is located at x = 153 and y = 41, in the center towards the top of the image, with a size = 3120 pixels. One build

In [24]:
messages = [{
    "role": "user", 
    "content": "Question 1: Given two short sides of a right triangle are 5 and 10, calculate its hypotenuse. Question 2: Given two short sides of a right triangle are 6 and 7, calculate its hypotenuse"
}]

functions = [
        {
            "name": "get_hypotenuse",
            "description": "Get the hypotenuse of a right triangle given its other two sides",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "string",
                        "description": "List of numbers. Each element represent the length of one short leg of a right triangle",
                    },
                    "b": {
                        "type": "string",
                        "description": "List of numbers. Each element represent the length of another short leg of a right triangle",
                    }
                },
                "required": ["a", "b"],
            },
        }
    ]


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]

In [25]:
response_message

<OpenAIObject at 0x1e183c85810> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"a\": \"5,6\",\n  \"b\": \"10,7\"\n}",
    "name": "get_hypotenuse"
  },
  "role": "assistant"
}

# DBSCAN

In [50]:
from sklearn.cluster import DBSCAN

In [38]:
train_labels_path = 'H:/xview/train_labels/xView_train.geojson'
annotations_path = "H:/xview/annotations"
train_images_path = "F:/train_images"
train_blocks_path = "F:/train_blocks"

In [40]:
with open("../results/unique_blocks_info.json", "r") as f:
    unique_blocks_info = json.load(f)

In [54]:
block_id = '104.tif_125'
polygons = get_polygons(unique_blocks_info[block_id], image_size=256)
X = [poly['rectangle_coordinates'] for poly in polygons["polygons"]]

In [55]:
X

[[-15, 22, 44, 74],
 [162, 29, 212, 77],
 [44, -34, 177, 54],
 [228, 44, 249, 65],
 [151, 89, 163, 99],
 [140, 99, 151, 109],
 [218, 25, 232, 40],
 [135, 124, 148, 138],
 [207, 11, 226, 28],
 [140, 117, 168, 153]]

In [56]:
def box_distance(array_a, array_b):
    x_a, y_a = (array_a[0] + array_a[2]) / 2, (array_a[1] + array_a[3]) / 2
    x_b, y_b = (array_b[0] + array_b[2]) / 2, (array_b[1] + array_b[3]) / 2
    
    return np.sqrt(np.square(x_a - x_b) + np.square(y_a - y_b))

In [57]:
clustering = DBSCAN(eps=3, min_samples=2, metric=box_distance).fit(X)

In [58]:
clustering.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int64)